In [1]:
# Goal is to convert x_train from (N x D) to (N x T x D) where T is the number of previous time steps.
# Then apply this N x T x D into RNN/LSTM models

In [2]:
# This RNN model considers 8 features for predictions of fututre DHI, DNI, GHI values
# It does not take into consideration past DHI, DNI, GHI values for evaluation

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

# Train excel sheet

In [4]:
train = pd.read_csv(r"C:\Users\ariha\Desktop\Hackathons & Projects\Wipro Hackathon\train.csv")
train

,Year,Month,Day,Hour,Minute,Clearsky DHI,Clearsky DNI,Clearsky GHI,Cloud Type,Dew Point,Temperature,Pressure,Relative Humidity,Solar Zenith Angle,Precipitable Water,Wind Direction,Wind Speed,Fill Flag
0,2009,1,1,0,0,0,0,0,0,0.0,5.0,1010,75.34,106.15,0.499,346.1,3.1,0
1,2009,1,1,0,30,0,0,0,0,1.0,5.0,1010,80.81,112.28,0.490,346.1,3.1,0
2,2009,1,1,1,0,0,0,0,4,0.0,5.0,1010,78.27,118.50,0.482,347.9,3.2,0
3,2009,1,1,1,30,0,0,0,4,0.0,4.0,1010,78.27,124.78,0.478,347.9,3.1,0
4,2009,1,1,2,0,0,0,0,4,0.0,4.0,1010,76.45,131.12,0.475,350.0,3.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175291,2018,12,31,21,30,51,555,168,4,19.4,20.8,1008,91.77,77.86,3.700,204.0,3.5,100
175292,2018,12,31,22,0,37,388,84,4,19.1,20.1,1008,93.88,83.03,3.800,209.0,3.2,100
175293,2018,12,31,22,30,15,115,18,7,19.1,19.6,1008,96.83,88.32,3.800,208.0,2.6,57
175294,2018,12,31,23,0,0,0,0,7,18.7,19.2,1009,96.84,94.34,3.700,206.0,2.1,0


In [5]:
# Creating a Date column
train['Date'] = pd.to_datetime(train[['Year', 'Month', 'Day', 'Hour', 'Minute']])
train.head()

,Year,Month,Day,Hour,Minute,Clearsky DHI,Clearsky DNI,Clearsky GHI,Cloud Type,Dew Point,Temperature,Pressure,Relative Humidity,Solar Zenith Angle,Precipitable Water,Wind Direction,Wind Speed,Fill Flag,Date
0,2009,1,1,0,0,0,0,0,0,0.0,5.0,1010,75.34,106.15,0.499,346.1,3.1,0,2009-01-01 00:00:00
1,2009,1,1,0,30,0,0,0,0,1.0,5.0,1010,80.81,112.28,0.490,346.1,3.1,0,2009-01-01 00:30:00
2,2009,1,1,1,0,0,0,0,4,0.0,5.0,1010,78.27,118.50,0.482,347.9,3.2,0,2009-01-01 01:00:00
3,2009,1,1,1,30,0,0,0,4,0.0,4.0,1010,78.27,124.78,0.478,347.9,3.1,0,2009-01-01 01:30:00
4,2009,1,1,2,0,0,0,0,4,0.0,4.0,1010,76.45,131.12,0.475,350.0,3.0,0,2009-01-01 02:00:00


In [6]:
train.drop(columns=['Year', 'Month', 'Day', 'Hour', 'Minute'], axis=1, inplace=True)
train.head()

,Clearsky DHI,Clearsky DNI,Clearsky GHI,Cloud Type,Dew Point,Temperature,Pressure,Relative Humidity,Solar Zenith Angle,Precipitable Water,Wind Direction,Wind Speed,Fill Flag,Date
0,0,0,0,0,0.0,5.0,1010,75.34,106.15,0.499,346.1,3.1,0,2009-01-01 00:00:00
1,0,0,0,0,1.0,5.0,1010,80.81,112.28,0.490,346.1,3.1,0,2009-01-01 00:30:00
2,0,0,0,4,0.0,5.0,1010,78.27,118.50,0.482,347.9,3.2,0,2009-01-01 01:00:00
3,0,0,0,4,0.0,4.0,1010,78.27,124.78,0.478,347.9,3.1,0,2009-01-01 01:30:00
4,0,0,0,4,0.0,4.0,1010,76.45,131.12,0.475,350.0,3.0,0,2009-01-01 02:00:00


In [7]:
train.shape

(175296, 14)

In [8]:
columns_list = []

for cols in train.columns:
    columns_list.append(cols)

columns_list

['Clearsky DHI',
 'Clearsky DNI',
 'Clearsky GHI',
 'Cloud Type',
 'Dew Point',
 'Temperature',
 'Pressure',
 'Relative Humidity',
 'Solar Zenith Angle',
 'Precipitable Water',
 'Wind Direction',
 'Wind Speed',
 'Fill Flag',
 'Date']

In [9]:
columns_list.pop()
columns_list.insert(0, 'Date')
columns_list

['Date',
 'Clearsky DHI',
 'Clearsky DNI',
 'Clearsky GHI',
 'Cloud Type',
 'Dew Point',
 'Temperature',
 'Pressure',
 'Relative Humidity',
 'Solar Zenith Angle',
 'Precipitable Water',
 'Wind Direction',
 'Wind Speed',
 'Fill Flag']

In [10]:
train = pd.DataFrame(train, columns=columns_list)
train.head()

,Date,Clearsky DHI,Clearsky DNI,Clearsky GHI,Cloud Type,Dew Point,Temperature,Pressure,Relative Humidity,Solar Zenith Angle,Precipitable Water,Wind Direction,Wind Speed,Fill Flag
0,2009-01-01 00:00:00,0,0,0,0,0.0,5.0,1010,75.34,106.15,0.499,346.1,3.1,0
1,2009-01-01 00:30:00,0,0,0,0,1.0,5.0,1010,80.81,112.28,0.490,346.1,3.1,0
2,2009-01-01 01:00:00,0,0,0,4,0.0,5.0,1010,78.27,118.50,0.482,347.9,3.2,0
3,2009-01-01 01:30:00,0,0,0,4,0.0,4.0,1010,78.27,124.78,0.478,347.9,3.1,0
4,2009-01-01 02:00:00,0,0,0,4,0.0,4.0,1010,76.45,131.12,0.475,350.0,3.0,0


In [11]:
corr_matrix = train.corr()
corr_matrix

,Clearsky DHI,Clearsky DNI,Clearsky GHI,Cloud Type,Dew Point,Temperature,Pressure,Relative Humidity,Solar Zenith Angle,Precipitable Water,Wind Direction,Wind Speed,Fill Flag
Clearsky DHI,1.000000,0.860988,0.912991,0.022479,0.222931,0.503409,-0.093766,-0.559766,-0.873421,0.207504,-0.000195,0.315655,0.000740
Clearsky DNI,0.860988,1.000000,0.927926,-0.073302,0.033601,0.384353,0.007043,-0.680471,-0.886323,-0.002180,0.006256,0.381162,0.006318
Clearsky GHI,0.912991,0.927926,1.000000,-0.047297,0.140420,0.478273,-0.050556,-0.655765,-0.891033,0.108725,0.004944,0.349180,-0.030297
Cloud Type,0.022479,-0.073302,-0.047297,1.000000,0.202096,0.066777,-0.137158,0.235374,0.026873,0.326904,-0.081565,0.072632,0.090161
Dew Point,0.222931,0.033601,0.140420,0.202096,1.000000,0.848948,-0.419534,0.193455,-0.222336,0.881881,-0.076357,-0.140699,0.036851
Temperature,0.503409,0.384353,0.478273,0.066777,0.848948,1.000000,-0.371395,-0.341914,-0.526548,0.742449,-0.066673,-0.046694,0.015734
Pressure,-0.093766,0.007043,-0.050556,-0.137158,-0.419534,-0.371395,1.000000,-0.085004,0.078900,-0.365342,-0.182006,-0.122674,0.052521
Relative Humidity,-0.559766,-0.680471,-0.655765,0.235374,0.193455,-0.341914,-0.085004,1.000000,0.607108,0.182658,-0.019108,-0.172248,0.023674
Solar Zenith Angle,-0.873421,-0.886323,-0.891033,0.026873,-0.222336,-0.526548,0.078900,0.607108,1.000000,-0.182529,0.003802,-0.295927,-0.029150
Precipitable Water,0.207504,-0.002180,0.108725,0.326904,0.881881,0.742449,-0.365342,0.182658,-0.182529,1.000000,-0.105735,-0.106892,0.044106


In [12]:
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
upper

C:\Users\ariha\AppData\Local\Temp/ipykernel_10640/2621372733.py:1: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))


,Clearsky DHI,Clearsky DNI,Clearsky GHI,Cloud Type,Dew Point,Temperature,Pressure,Relative Humidity,Solar Zenith Angle,Precipitable Water,Wind Direction,Wind Speed,Fill Flag
Clearsky DHI,NaN,0.860988,0.912991,0.022479,0.222931,0.503409,-0.093766,-0.559766,-0.873421,0.207504,-0.000195,0.315655,0.000740
Clearsky DNI,NaN,NaN,0.927926,-0.073302,0.033601,0.384353,0.007043,-0.680471,-0.886323,-0.002180,0.006256,0.381162,0.006318
Clearsky GHI,NaN,NaN,NaN,-0.047297,0.140420,0.478273,-0.050556,-0.655765,-0.891033,0.108725,0.004944,0.349180,-0.030297
Cloud Type,NaN,NaN,NaN,NaN,0.202096,0.066777,-0.137158,0.235374,0.026873,0.326904,-0.081565,0.072632,0.090161
Dew Point,NaN,NaN,NaN,NaN,NaN,0.848948,-0.419534,0.193455,-0.222336,0.881881,-0.076357,-0.140699,0.036851
Temperature,NaN,NaN,NaN,NaN,NaN,NaN,-0.371395,-0.341914,-0.526548,0.742449,-0.066673,-0.046694,0.015734
Pressure,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.085004,0.078900,-0.365342,-0.182006,-0.122674,0.052521
Relative Humidity,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.607108,0.182658,-0.019108,-0.172248,0.023674
Solar Zenith Angle,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.182529,0.003802,-0.295927,-0.029150
Precipitable Water,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.105735,-0.106892,0.044106


In [13]:
columns_to_drop = []
for cols in upper.columns:
    if any (upper[cols] > 0.75):
        columns_to_drop.append(cols)
columns_to_drop

['Clearsky DNI', 'Clearsky GHI', 'Temperature', 'Precipitable Water']

In [14]:
train.drop(columns=['Temperature', 'Precipitable Water'], axis=1, inplace=True)
train.head()

,Date,Clearsky DHI,Clearsky DNI,Clearsky GHI,Cloud Type,Dew Point,Pressure,Relative Humidity,Solar Zenith Angle,Wind Direction,Wind Speed,Fill Flag
0,2009-01-01 00:00:00,0,0,0,0,0.0,1010,75.34,106.15,346.1,3.1,0
1,2009-01-01 00:30:00,0,0,0,0,1.0,1010,80.81,112.28,346.1,3.1,0
2,2009-01-01 01:00:00,0,0,0,4,0.0,1010,78.27,118.50,347.9,3.2,0
3,2009-01-01 01:30:00,0,0,0,4,0.0,1010,78.27,124.78,347.9,3.1,0
4,2009-01-01 02:00:00,0,0,0,4,0.0,1010,76.45,131.12,350.0,3.0,0


In [15]:
train.shape

(175296, 12)

In [16]:
y = train.iloc[:, 1:4]
y.head()

,Clearsky DHI,Clearsky DNI,Clearsky GHI
0,0,0,0
1,0,0,0
2,0,0,0
3,0,0,0
4,0,0,0


In [17]:
y.shape

(175296, 3)

In [18]:
x = train.iloc[:, 4:]
x.head()

,Cloud Type,Dew Point,Pressure,Relative Humidity,Solar Zenith Angle,Wind Direction,Wind Speed,Fill Flag
0,0,0.0,1010,75.34,106.15,346.1,3.1,0
1,0,1.0,1010,80.81,112.28,346.1,3.1,0
2,4,0.0,1010,78.27,118.50,347.9,3.2,0
3,4,0.0,1010,78.27,124.78,347.9,3.1,0
4,4,0.0,1010,76.45,131.12,350.0,3.0,0


In [19]:
x.shape

(175296, 8)

In [20]:
scaler_x = MinMaxScaler((0, 1))
scaler_y = MinMaxScaler((0, 1))

x = scaler_x.fit_transform(x)
y = scaler_y.fit_transform(y)

In [21]:
x.shape[0]

175296

In [22]:
y.shape

(175296, 3)

In [23]:
x_train = []
y_train = []

for i in range(2, x.shape[0]):
    x_train.append(x[i-2 : i])
    y_train.append(y[i])

In [24]:
len(x_train), len(x_train[0]), len(x_train[0][0])

(175294, 2, 8)

In [25]:
len(y_train), len(y_train[0])

(175294, 3)

In [26]:
x_train = np.array(x_train)
y_train = np.array(y_train)

In [27]:
x_train.shape, y_train.shape

((175294, 2, 8), (175294, 3))

In [28]:
len(x_train)

175294

In [29]:
x_val = np.array(x_train[int(len(x_train) * 0.90) :])
y_val = np.array(y_train[int(len(y_train) * 0.90) :])

x_val.shape, y_val.shape

((17530, 2, 8), (17530, 3))

In [30]:
x_train = np.array(x_train[: int(len(x_train) * 0.90)])
y_train = np.array(y_train[: int(len(y_train) * 0.90)])

x_train.shape, y_train.shape

((157764, 2, 8), (157764, 3))

# Test Excel sheet

In [31]:
test = pd.read_csv(r"C:\Users\ariha\Desktop\Hackathons & Projects\Wipro Hackathon\test sheet for RNNs and LSTMs.csv")
test

,Year,Month,Day,Hour,Minute,Cloud Type,Dew Point,Temperature,Pressure,Relative Humidity,Solar Zenith Angle,Precipitable Water,Wind Direction,Wind Speed,Fill Flag,Clearsky DHI,Clearsky DNI,Clearsky GHI
0,2018,12,31,23,0,7,18.7,19.2,1009,96.84,94.34,3.7,206,2.1,0,0.0,0.0,0.0
1,2018,12,31,23,30,7,18.7,19.2,1009,96.84,100.22,3.7,206,2.1,0,0.0,0.0,0.0
2,2019,1,1,0,0,7,18.4,18.8,1008,97.70,106.23,3.5,190,2.3,0,NaN,NaN,NaN
3,2019,1,1,0,30,3,18.4,18.6,1008,98.92,112.36,3.5,187,2.5,0,NaN,NaN,NaN
4,2019,1,1,1,0,3,18.2,18.5,1008,98.35,118.58,3.5,184,2.8,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17517,2019,12,31,21,30,0,5.6,11.1,1006,68.64,77.89,0.8,276,2.2,0,NaN,NaN,NaN
17518,2019,12,31,22,0,0,5.7,9.3,1006,78.31,83.06,0.8,274,1.7,0,NaN,NaN,NaN
17519,2019,12,31,22,30,0,5.7,8.5,1006,82.64,88.34,0.8,273,1.8,0,NaN,NaN,NaN
17520,2019,12,31,23,0,0,4.8,7.8,1006,81.49,94.37,0.8,272,1.8,0,NaN,NaN,NaN


In [32]:
test.drop(test.iloc[:, -3:], axis=1, inplace=True)
test.head()

,Year,Month,Day,Hour,Minute,Cloud Type,Dew Point,Temperature,Pressure,Relative Humidity,Solar Zenith Angle,Precipitable Water,Wind Direction,Wind Speed,Fill Flag
0,2018,12,31,23,0,7,18.7,19.2,1009,96.84,94.34,3.7,206,2.1,0
1,2018,12,31,23,30,7,18.7,19.2,1009,96.84,100.22,3.7,206,2.1,0
2,2019,1,1,0,0,7,18.4,18.8,1008,97.70,106.23,3.5,190,2.3,0
3,2019,1,1,0,30,3,18.4,18.6,1008,98.92,112.36,3.5,187,2.5,0
4,2019,1,1,1,0,3,18.2,18.5,1008,98.35,118.58,3.5,184,2.8,0


In [33]:
test['Date'] = pd.to_datetime(test[['Year', 'Month', 'Day', 'Hour', 'Minute']])
test.head()

,Year,Month,Day,Hour,Minute,Cloud Type,Dew Point,Temperature,Pressure,Relative Humidity,Solar Zenith Angle,Precipitable Water,Wind Direction,Wind Speed,Fill Flag,Date
0,2018,12,31,23,0,7,18.7,19.2,1009,96.84,94.34,3.7,206,2.1,0,2018-12-31 23:00:00
1,2018,12,31,23,30,7,18.7,19.2,1009,96.84,100.22,3.7,206,2.1,0,2018-12-31 23:30:00
2,2019,1,1,0,0,7,18.4,18.8,1008,97.70,106.23,3.5,190,2.3,0,2019-01-01 00:00:00
3,2019,1,1,0,30,3,18.4,18.6,1008,98.92,112.36,3.5,187,2.5,0,2019-01-01 00:30:00
4,2019,1,1,1,0,3,18.2,18.5,1008,98.35,118.58,3.5,184,2.8,0,2019-01-01 01:00:00


In [34]:
test.drop(test.iloc[:, :5], axis=1, inplace=True)
test.head()

,Cloud Type,Dew Point,Temperature,Pressure,Relative Humidity,Solar Zenith Angle,Precipitable Water,Wind Direction,Wind Speed,Fill Flag,Date
0,7,18.7,19.2,1009,96.84,94.34,3.7,206,2.1,0,2018-12-31 23:00:00
1,7,18.7,19.2,1009,96.84,100.22,3.7,206,2.1,0,2018-12-31 23:30:00
2,7,18.4,18.8,1008,97.70,106.23,3.5,190,2.3,0,2019-01-01 00:00:00
3,3,18.4,18.6,1008,98.92,112.36,3.5,187,2.5,0,2019-01-01 00:30:00
4,3,18.2,18.5,1008,98.35,118.58,3.5,184,2.8,0,2019-01-01 01:00:00


In [35]:
columns_list_test = []

for cols in test.columns:
    columns_list_test.append(cols)
columns_list_test

['Cloud Type',
 'Dew Point',
 'Temperature',
 'Pressure',
 'Relative Humidity',
 'Solar Zenith Angle',
 'Precipitable Water',
 'Wind Direction',
 'Wind Speed',
 'Fill Flag',
 'Date']

In [36]:
columns_list_test.pop()
columns_list_test.insert(0, 'Date')

In [37]:
columns_list_test

['Date',
 'Cloud Type',
 'Dew Point',
 'Temperature',
 'Pressure',
 'Relative Humidity',
 'Solar Zenith Angle',
 'Precipitable Water',
 'Wind Direction',
 'Wind Speed',
 'Fill Flag']

In [38]:
test = pd.DataFrame(test, columns=columns_list_test)
test.head()

,Date,Cloud Type,Dew Point,Temperature,Pressure,Relative Humidity,Solar Zenith Angle,Precipitable Water,Wind Direction,Wind Speed,Fill Flag
0,2018-12-31 23:00:00,7,18.7,19.2,1009,96.84,94.34,3.7,206,2.1,0
1,2018-12-31 23:30:00,7,18.7,19.2,1009,96.84,100.22,3.7,206,2.1,0
2,2019-01-01 00:00:00,7,18.4,18.8,1008,97.70,106.23,3.5,190,2.3,0
3,2019-01-01 00:30:00,3,18.4,18.6,1008,98.92,112.36,3.5,187,2.5,0
4,2019-01-01 01:00:00,3,18.2,18.5,1008,98.35,118.58,3.5,184,2.8,0


In [39]:
test.drop(columns=['Temperature', 'Precipitable Water'], axis=1, inplace=True)
test.head()

,Date,Cloud Type,Dew Point,Pressure,Relative Humidity,Solar Zenith Angle,Wind Direction,Wind Speed,Fill Flag
0,2018-12-31 23:00:00,7,18.7,1009,96.84,94.34,206,2.1,0
1,2018-12-31 23:30:00,7,18.7,1009,96.84,100.22,206,2.1,0
2,2019-01-01 00:00:00,7,18.4,1008,97.70,106.23,190,2.3,0
3,2019-01-01 00:30:00,3,18.4,1008,98.92,112.36,187,2.5,0
4,2019-01-01 01:00:00,3,18.2,1008,98.35,118.58,184,2.8,0


In [40]:
test.shape

(17522, 9)

In [41]:
x_test_df = test.iloc[:, 1:]
x_test_df.head()

,Cloud Type,Dew Point,Pressure,Relative Humidity,Solar Zenith Angle,Wind Direction,Wind Speed,Fill Flag
0,7,18.7,1009,96.84,94.34,206,2.1,0
1,7,18.7,1009,96.84,100.22,206,2.1,0
2,7,18.4,1008,97.70,106.23,190,2.3,0
3,3,18.4,1008,98.92,112.36,187,2.5,0
4,3,18.2,1008,98.35,118.58,184,2.8,0


In [42]:
x_test_scaled = scaler_x.fit_transform(x_test_df)
x_test_scaled

array([[0.77777778, 0.79651163, 0.6       , ..., 0.57222222, 0.24691358,
        0.        ],
       [0.77777778, 0.79651163, 0.6       , ..., 0.57222222, 0.24691358,
        0.        ],
       [0.77777778, 0.7877907 , 0.57142857, ..., 0.52777778, 0.27160494,
        0.        ],
       ...,
       [0.        , 0.41860465, 0.51428571, ..., 0.75833333, 0.20987654,
        0.        ],
       [0.        , 0.39244186, 0.51428571, ..., 0.75555556, 0.20987654,
        0.        ],
       [0.        , 0.39244186, 0.51428571, ..., 0.75555556, 0.20987654,
        0.        ]])

In [43]:
x_test_scaled.shape[0]

17522

In [44]:
x_test = []

for i in range(2, x_test_scaled.shape[0]):
    x_test.append(x_test_scaled[i-2 : i])

In [45]:
len(x_test), len(x_test[0]), len(x_test[0][0])

(17520, 2, 8)

In [46]:
x_test = np.array(x_test)
x_test.shape

(17520, 2, 8)

# Making sure all arrays are of the right shape

In [47]:
x_train.shape

(157764, 2, 8)

In [48]:
y_train.shape

(157764, 3)

In [49]:
x_val.shape

(17530, 2, 8)

In [50]:
y_val.shape

(17530, 3)

In [51]:
x_test.shape

(17520, 2, 8)

# Creating RNN model

In [52]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import SimpleRNN, LSTM, Dense, Input, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

In [53]:
i = Input(shape=x_train[0].shape)
x = LSTM(256) (i)
x = Dropout(0.2) (x)
x = Dense(3) (x)

model = Model(i, x)

In [54]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 2, 8)]            0         
                                                                 
 lstm (LSTM)                 (None, 256)               271360    
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense (Dense)               (None, 3)                 771       
                                                                 
Total params: 272,131
Trainable params: 272,131
Non-trainable params: 0
_________________________________________________________________


In [55]:
model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')

In [56]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10, restore_best_weights=True)
r = model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=100, callbacks=[es])

Epoch 1/100
4931/4931 [==============================] - 35s 4ms/step - loss: 0.0073 - val_loss: 0.0296
Epoch 2/100
4931/4931 [==============================] - 16s 3ms/step - loss: 0.0026 - val_loss: 0.0214
Epoch 3/100
4931/4931 [==============================] - 20s 4ms/step - loss: 0.0018 - val_loss: 0.0039
Epoch 4/100
4931/4931 [==============================] - 16s 3ms/step - loss: 0.0016 - val_loss: 0.0023
Epoch 5/100
4931/4931 [==============================] - 16s 3ms/step - loss: 0.0016 - val_loss: 0.0022
Epoch 6/100
4931/4931 [==============================] - 17s 3ms/step - loss: 0.0015 - val_loss: 0.0019
Epoch 7/100
4931/4931 [==============================] - 16s 3ms/step - loss: 0.0015 - val_loss: 0.0024
Epoch 8/100
4931/4931 [==============================] - 16s 3ms/step - loss: 0.0015 - val_loss: 0.0025
Epoch 9/100
4931/4931 [==============================] - 16s 3ms/step - loss: 0.0014 - val_loss: 0.0024
Epoch 10/100
4931/4931 [==============================] - 16s 3m

In [57]:
y_pred = model.predict(x_test)

In [58]:
y_pred

array([[ 7.1112439e-03,  2.3239255e-02,  1.1651799e-02],
       [ 1.6862229e-03,  4.8983693e-03,  4.1227192e-03],
       [-1.1306778e-03, -8.9491606e-03,  9.6574426e-05],
       ...,
       [ 2.0670809e-02,  1.5142891e-01,  3.2890663e-02],
       [-2.1984354e-03,  7.0209265e-02,  1.0957792e-02],
       [-1.4244020e-02,  1.5867770e-02, -3.0731410e-03]], dtype=float32)

In [59]:
len(y_pred), len(y_pred[0])

(17520, 3)

In [60]:
scaler_y.scale_

array([0.00176991, 0.00096154, 0.00094073])

In [61]:
y_pred = scaler_y.inverse_transform(y_pred)
y_pred

array([[ 4.01785278e+00,  2.41688251e+01,  1.23858624e+01],
       [ 9.52715933e-01,  5.09430408e+00,  4.38245058e+00],
       [-6.38832927e-01, -9.30712700e+00,  1.02658615e-01],
       ...,
       [ 1.16790066e+01,  1.57486069e+02,  3.49627762e+01],
       [-1.24211597e+00,  7.30176392e+01,  1.16481333e+01],
       [-8.04787159e+00,  1.65024796e+01, -3.26674891e+00]], dtype=float32)

In [62]:
len(y_pred)

17520

In [63]:
new_y_pred = []
for rows in y_pred:
    for element in rows:
        if element < 0:
            element = 0
        new_y_pred.append(element)
len(new_y_pred)

52560

In [64]:
y_pred = np.array(new_y_pred).reshape(len(y_pred), 3)
y_pred

array([[4.01785278e+00, 2.41688251e+01, 1.23858624e+01],
       [9.52715933e-01, 5.09430408e+00, 4.38245058e+00],
       [0.00000000e+00, 0.00000000e+00, 1.02658615e-01],
       ...,
       [1.16790066e+01, 1.57486069e+02, 3.49627762e+01],
       [0.00000000e+00, 7.30176392e+01, 1.16481333e+01],
       [0.00000000e+00, 1.65024796e+01, 0.00000000e+00]])

In [65]:
y_pred = pd.DataFrame(y_pred, columns=['Clearsky DHI', 'Clearsky DNI', 'Clearsky GHI'])
y_pred

,Clearsky DHI,Clearsky DNI,Clearsky GHI
0,4.017853,24.168825,12.385862
1,0.952716,5.094304,4.382451
2,0.000000,0.000000,0.102659
3,0.397026,0.000000,5.951111
4,1.651405,5.070252,11.990910
...,...,...,...
17515,46.943134,601.716553,168.659088
17516,31.312468,403.827454,93.059883
17517,11.679007,157.486069,34.962776
17518,0.000000,73.017639,11.648133


In [66]:
y_pred.to_csv('submission_file-LSTM.csv', index=False)